# Chapter 3: The Algorithm Zoo 🦁

Welcome to the Algorithm Zoo! Just like you pick the right data structure for the job (ArrayList vs HashMap vs Tree), you pick the right ML algorithm.

This notebook covers:
1. **Linear Regression** — The Array (simple, fast, rigid)
2. **Decision Trees** — The If/Else Block (interpretable, prone to overfitting)
3. **Random Forests** — The Distributed System (robust, best default)

Each section includes **interactive visualizations** to build intuition.

In [ ]:
# Standard imports for the entire notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

# Set a clean style for all plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 11

---
## 1. Linear Regression (The Array)

**The Concept**: Draw a straight line through the data.

**SWE Analogy**: Like an **Array**—extremely fast and simple, but rigid. If your data isn't linear, it fails.

Let's visualize what "fitting a line" actually means.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Generate synthetic house price data
np.random.seed(42)
n_samples = 100

# Square footage (1000 - 3000 sqft)
sqft = np.random.uniform(1000, 3000, n_samples)

# Price = base + (price_per_sqft * sqft) + noise
# Realistic: $100/sqft + $50k base
price = 50000 + 100 * sqft + np.random.normal(0, 20000, n_samples)

# Reshape for sklearn
X = sqft.reshape(-1, 1)
y = price

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

In [ ]:
# Train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# The "learned" parameters
print(f"Learned Formula: Price = ${lr_model.intercept_:,.0f} + ${lr_model.coef_[0]:.2f} × SqFt")
print(f"\nInterpretation: Base price ~${lr_model.intercept_:,.0f}, plus ~${lr_model.coef_[0]:.0f} per square foot")

### Visualization 1: The Fitted Line
This is what "training" looks like—finding the line that best fits the data.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: The fitted line
ax1 = axes[0]
ax1.scatter(X_train, y_train, alpha=0.6, c='steelblue', label='Training Data', edgecolors='white')
ax1.scatter(X_test, y_test, alpha=0.6, c='coral', label='Test Data', edgecolors='white')

# Draw the regression line
x_line = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_line = lr_model.predict(x_line)
ax1.plot(x_line, y_line, 'k-', linewidth=2, label='Fitted Line')

ax1.set_xlabel('Square Footage')
ax1.set_ylabel('House Price ($)')
ax1.set_title('Linear Regression: Fitting a Line Through Data')
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))

# Plot 2: Actual vs Predicted
ax2 = axes[1]
y_pred = lr_model.predict(X_test)
ax2.scatter(y_test, y_pred, alpha=0.7, c='teal', edgecolors='white', s=60)

# Perfect prediction line
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

ax2.set_xlabel('Actual Price ($)')
ax2.set_ylabel('Predicted Price ($)')
ax2.set_title('Actual vs Predicted (closer to line = better)')
ax2.legend()
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))

plt.tight_layout()
plt.show()

print(f"\nR² Score: {r2_score(y_test, y_pred):.3f} (1.0 = perfect, 0 = useless)")

### Visualization 2: Residuals (The Errors)
Residuals show where the model is wrong. Ideally, errors should be random—no pattern.

In [ ]:
residuals = y_test - y_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Predicted
ax1 = axes[0]
ax1.scatter(y_pred, residuals, alpha=0.7, c='purple', edgecolors='white', s=60)
ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Predicted Price')
ax1.set_ylabel('Residual (Error)')
ax1.set_title('Residuals Plot: Random Scatter = Good Model')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}k'))

# Histogram of residuals
ax2 = axes[1]
ax2.hist(residuals, bins=15, color='mediumpurple', edgecolor='white', alpha=0.8)
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Residual (Error)')
ax2.set_ylabel('Frequency')
ax2.set_title('Residual Distribution: Should Be Centered at 0')

plt.tight_layout()
plt.show()

### Visualization 3: When Linear Regression Fails
What happens when the data isn't actually linear?

In [ ]:
# Generate non-linear data (quadratic relationship)
np.random.seed(42)
x_nonlinear = np.linspace(0, 10, 100)
y_nonlinear = 2 * x_nonlinear**2 - 5 * x_nonlinear + 10 + np.random.normal(0, 8, 100)

# Fit linear regression to non-linear data
lr_bad = LinearRegression()
lr_bad.fit(x_nonlinear.reshape(-1, 1), y_nonlinear)
y_bad_pred = lr_bad.predict(x_nonlinear.reshape(-1, 1))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Good fit (linear data)
ax1 = axes[0]
ax1.scatter(X, y, alpha=0.5, c='steelblue', edgecolors='white')
ax1.plot(x_line, y_line, 'green', linewidth=3, label='Linear Fit')
ax1.set_title('✓ Linear Data → Linear Model Works!')
ax1.set_xlabel('Square Footage')
ax1.set_ylabel('Price')
ax1.legend()

# Bad fit (non-linear data)
ax2 = axes[1]
ax2.scatter(x_nonlinear, y_nonlinear, alpha=0.5, c='coral', edgecolors='white')
ax2.plot(x_nonlinear, y_bad_pred, 'red', linewidth=3, label='Linear Fit (Bad!)')
ax2.set_title('✗ Curved Data → Linear Model Fails!')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.legend()

plt.tight_layout()
plt.show()

print("Lesson: Linear Regression is like an Array—fast and simple, but only works for linear patterns.")

---
## 2. Decision Trees (The If/Else Block)

**The Concept**: A flowchart that asks yes/no questions to classify data.

**SWE Analogy**: A giant **nested if/else statement**. Highly readable, but can get overfit (too specific).

Let's visualize how a tree "thinks."

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Load the Iris dataset (classic ML dataset)
iris = load_iris()
X_iris = iris.data[:, :2]  # Use only first 2 features for visualization
y_iris = iris.target
feature_names = iris.feature_names[:2]
class_names = iris.target_names

# Split
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=42
)

print(f"Dataset: {len(X_iris)} flowers, {len(class_names)} species")
print(f"Features: {feature_names}")
print(f"Classes: {list(class_names)}")

In [ ]:
# Train a small tree (max_depth=3 for readability)
tree_model = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_model.fit(X_train_iris, y_train_iris)

tree_preds = tree_model.predict(X_test_iris)
print(f"Decision Tree Accuracy: {accuracy_score(y_test_iris, tree_preds):.2%}")

### Visualization 1: The Tree Structure
This is literally the "code" the model learned—a series of if/else conditions!

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
plot_tree(
    tree_model, 
    feature_names=feature_names, 
    class_names=class_names,
    filled=True, 
    rounded=True,
    ax=ax,
    fontsize=10
)
plt.title('Decision Tree: Literally a Flowchart!', fontsize=14)
plt.tight_layout()
plt.show()

print("Read it like code: 'IF petal width <= 0.8 THEN setosa, ELSE check petal length...'")

### Visualization 2: Decision Boundaries
This shows HOW the tree divides the feature space. Each color = a predicted class.

In [ ]:
def plot_decision_boundary(model, X, y, ax, title, class_names=None):
    """Helper function to plot decision boundaries"""
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ['#FF0000', '#00FF00', '#0000FF']
    
    # Create mesh grid
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 200),
        np.linspace(y_min, y_max, 200)
    )
    
    # Predict on mesh
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot
    ax.contourf(xx, yy, Z, alpha=0.4, cmap=cmap_light)
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=ListedColormap(cmap_bold), 
                         edgecolors='white', s=50, alpha=0.8)
    ax.set_title(title)
    
    if class_names is not None:
        handles = [plt.scatter([], [], c=cmap_bold[i], label=class_names[i], s=50) 
                   for i in range(len(class_names))]
        ax.legend(handles=handles, loc='upper left')
    
    return ax

# Plot decision boundary
fig, ax = plt.subplots(figsize=(10, 8))
plot_decision_boundary(tree_model, X_iris, y_iris, ax, 
                      'Decision Tree Boundaries (max_depth=3)',
                      class_names)
ax.set_xlabel(feature_names[0])
ax.set_ylabel(feature_names[1])
plt.tight_layout()
plt.show()

print("Notice the RECTANGULAR boundaries—trees only make axis-aligned splits.")

### Visualization 3: Overfitting Demo
What happens when we remove the `max_depth` limit?

In [ ]:
# Train trees with different depths
depths = [1, 3, 5, None]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for ax, depth in zip(axes.flat, depths):
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train_iris, y_train_iris)
    
    train_acc = accuracy_score(y_train_iris, tree.predict(X_train_iris))
    test_acc = accuracy_score(y_test_iris, tree.predict(X_test_iris))
    
    depth_str = str(depth) if depth else 'Unlimited'
    title = f'max_depth={depth_str}\nTrain: {train_acc:.1%} | Test: {test_acc:.1%}'
    plot_decision_boundary(tree, X_iris, y_iris, ax, title)
    ax.set_xlabel(feature_names[0])
    ax.set_ylabel(feature_names[1])

plt.tight_layout()
plt.show()

print("\n⚠️ OVERFITTING: Unlimited depth = 100% training accuracy but worse test accuracy!")
print("The model memorized the training data instead of learning general patterns.")

---
## 3. Random Forests (The Distributed System)

**The Concept**: Train 100 different trees on random subsets of data. Let them **vote**.

**SWE Analogy**: Like **RAID** or **distributed consensus**—many weak components become one reliable system.

Let's see the "wisdom of the crowd" in action.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf_model.fit(X_train_iris, y_train_iris)

rf_preds = rf_model.predict(X_test_iris)
print(f"Random Forest Accuracy: {accuracy_score(y_test_iris, rf_preds):.2%}")

### Visualization 1: How Trees Vote
Each tree in the forest makes a prediction. The final answer = majority vote.

In [ ]:
# Get predictions from individual trees for a single sample
sample_idx = 0
sample = X_test_iris[sample_idx].reshape(1, -1)
true_label = y_test_iris[sample_idx]

# Get each tree's prediction
individual_predictions = np.array([tree.predict(sample)[0] for tree in rf_model.estimators_])

# Count votes
vote_counts = [np.sum(individual_predictions == i) for i in range(3)]

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of votes
ax1 = axes[0]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = ax1.bar(class_names, vote_counts, color=colors, edgecolor='white', linewidth=2)
ax1.set_ylabel('Number of Trees')
ax1.set_xlabel('Predicted Class')
ax1.set_title(f'100 Trees Vote on Sample #{sample_idx}\nTrue Label: {class_names[true_label]}')
for bar, count in zip(bars, vote_counts):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{count}', ha='center', fontsize=12, fontweight='bold')

# Show first 20 trees' individual votes
ax2 = axes[1]
first_20 = individual_predictions[:20]
tree_colors = [colors[p] for p in first_20]
ax2.bar(range(20), [1]*20, color=tree_colors, edgecolor='white')
ax2.set_xlabel('Tree Index (first 20 of 100)')
ax2.set_ylabel('')
ax2.set_yticks([])
ax2.set_title('Each Bar = One Tree\'s Vote')

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors[i], label=class_names[i]) for i in range(3)]
ax2.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

final_pred = rf_model.predict(sample)[0]
print(f"\nFinal Prediction: {class_names[final_pred]} (won with {max(vote_counts)} votes)")

### Visualization 2: Decision Boundary Comparison
Compare single tree vs forest—notice how the forest is smoother?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Single deep tree (prone to overfitting)
single_tree = DecisionTreeClassifier(max_depth=10, random_state=42)
single_tree.fit(X_train_iris, y_train_iris)
single_acc = accuracy_score(y_test_iris, single_tree.predict(X_test_iris))
plot_decision_boundary(single_tree, X_iris, y_iris, axes[0], 
                      f'Single Tree (depth=10)\nTest Accuracy: {single_acc:.1%}',
                      class_names)

# Random Forest (robust)
rf_acc = accuracy_score(y_test_iris, rf_preds)
plot_decision_boundary(rf_model, X_iris, y_iris, axes[1], 
                      f'Random Forest (100 trees)\nTest Accuracy: {rf_acc:.1%}',
                      class_names)

for ax in axes:
    ax.set_xlabel(feature_names[0])
    ax.set_ylabel(feature_names[1])

plt.tight_layout()
plt.show()

print("Notice: Single tree has jagged, overfit boundaries. Forest is smoother and more general.")

### Visualization 3: Feature Importance
Random Forests can tell you WHICH features matter most.

In [ ]:
# Train on ALL features for better importance analysis
rf_full = RandomForestClassifier(n_estimators=100, random_state=42)
rf_full.fit(iris.data, iris.target)

# Get feature importances
importances = rf_full.feature_importances_
indices = np.argsort(importances)[::-1]

fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importances)))
bars = ax.barh(range(len(importances)), importances[indices], color=colors, edgecolor='white')
ax.set_yticks(range(len(importances)))
ax.set_yticklabels([iris.feature_names[i] for i in indices])
ax.set_xlabel('Importance Score')
ax.set_title('Feature Importance: Which Features Matter?')
ax.invert_yaxis()

# Add percentage labels
for bar, imp in zip(bars, importances[indices]):
    ax.text(imp + 0.01, bar.get_y() + bar.get_height()/2, 
            f'{imp:.1%}', va='center', fontsize=11)

plt.tight_layout()
plt.show()

print(f"\nTop feature: {iris.feature_names[indices[0]]} ({importances[indices[0]]:.1%} importance)")

---
## 4. The Polymorphism of Scikit-Learn

Notice how we never changed our evaluation code? That's the beauty of a consistent API.

```python
# The "Interface"
class Model:
    def fit(self, X, y): pass
    def predict(self, X): pass
```

You can swap algorithms like swapping data structures—same interface, different performance.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits

# Load digits dataset (from Chapter 2)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_digits, y_digits, test_size=0.2, random_state=42
)

# Define our "zoo" of models
models = {
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Same code, different models!
results = {}
for name, model in models.items():
    model.fit(X_train_d, y_train_d)  # Same interface
    preds = model.predict(X_test_d)  # Same interface
    accuracy = accuracy_score(y_test_d, preds)
    results[name] = accuracy
    print(f"{name}: {accuracy:.2%}")

### Final Visualization: Algorithm Showdown

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

names = list(results.keys())
accuracies = list(results.values())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

bars = ax.barh(names, accuracies, color=colors, edgecolor='white', height=0.6)
ax.set_xlim(0.8, 1.0)
ax.set_xlabel('Accuracy')
ax.set_title('Algorithm Showdown: Digits Recognition', fontsize=14)

# Add percentage labels
for bar, acc in zip(bars, accuracies):
    ax.text(acc + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{acc:.1%}', va='center', fontsize=12, fontweight='bold')

# Add SWE analogy labels
analogies = ['(HashMap lookup)', '(If/Else statements)', '(Distributed consensus)']
for i, (bar, analogy) in enumerate(zip(bars, analogies)):
    ax.text(0.81, bar.get_y() + bar.get_height()/2, 
            analogy, va='center', fontsize=9, style='italic', alpha=0.7)

plt.tight_layout()
plt.show()

winner = max(results, key=results.get)
print(f"\n🏆 Winner: {winner} with {results[winner]:.1%} accuracy")

---
## Summary: Your Algorithm Cheat Sheet

| Algorithm | SWE Analogy | When to Use | Trade-off |
|-----------|-------------|-------------|------------|
| **Linear Regression** | Array | Additive relationships (Price = A×SqFt + B) | Fast but rigid |
| **Decision Tree** | If/Else | Need explainability (loan approval) | Interpretable but overfits |
| **Random Forest** | RAID/Consensus | Default for tabular data | Robust but black-box |
| **K-Nearest Neighbors** | HashMap lookup | When similar inputs → similar outputs | Simple but slow at scale |

---
## ✦ Challenge

1. Go back to the **Decision Tree overfitting demo**. What `max_depth` gives the best test accuracy?
2. In the **Algorithm Showdown**, add `LogisticRegression` from `sklearn.linear_model`. How does it compare?
3. Try changing `n_estimators` in Random Forest from 100 to 10. Does accuracy drop significantly?

**Next Chapter**: We have multiple models—but how do we *really* know which is best? Accuracy can lie. We'll explore **Evaluation Metrics**—the unit tests of ML.